# 載入需要資料/library

In [1]:
%env KERAS_BACKEND = tensorflow

env: KERAS_BACKEND=tensorflow


In [21]:
import keras
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Lambda, concatenate
from keras.optimizers import SGD,Adam
from keras.layers import Input
from keras.datasets import mnist
from keras.utils import np_utils
from keras import metrics
from keras import backend as K

In [22]:
(x_train0 , y_train0) , (x_test0 , y_test0) = mnist.load_data()

In [23]:
x_train = x_train0.reshape(60000,-1)
x_test = x_test0.reshape(10000,-1)

x_train = x_train/255
x_test = x_test/255

# 模型建立

In [24]:
x = Input(shape=(784,))
enc_1 = Dense(150, activation = 'sigmoid')
enc_2 = Dense(2, activation='sigmoid')

enc_mean = Dense(2)
enc_log_var = Dense(2)
h = enc_2(enc_1(x))

dec_2 = Dense(150, activation = 'sigmoid')
dec_1 = Dense(784, activation = 'sigmoid')

x_hat = dec_1(dec_2(h))

autoencoder = Model(x, x_hat)
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 784)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 150)               117750    
_________________________________________________________________
dense_14 (Dense)             (None, 2)                 302       
_________________________________________________________________
dense_17 (Dense)             (None, 150)               450       
_________________________________________________________________
dense_18 (Dense)             (None, 784)               118384    
Total params: 236,886
Trainable params: 236,886
Non-trainable params: 0
_________________________________________________________________


In [25]:
enc_x = enc_1(x)

z_mean = enc_mean(enc_x)
z_log_var = enc_log_var(enc_x)

In [26]:
def sampling(args):
    z_mean , z_log_var = args
    epsilon = K.random_normal(shape=(2,),mean=0. , stddev=1)
    return z_mean + K.exp(z_log_var / 2) * epsilon

In [27]:
z = Lambda(sampling, output_shape=(2,))([z_mean, z_log_var])
dec_x = dec_2(z)
x_hat = dec_1(dec_x)

VAE = Model(x, x_hat)
VAE.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 784)          0                                            
__________________________________________________________________________________________________
dense_13 (Dense)                (None, 150)          117750      input_3[0][0]                    
__________________________________________________________________________________________________
dense_15 (Dense)                (None, 2)            302         dense_13[1][0]                   
__________________________________________________________________________________________________
dense_16 (Dense)                (None, 2)            302         dense_13[1][0]                   
__________________________________________________________________________________________________
lambda_2 (

In [ ]:
def vae_loss(x, x_recon):  
    recovery_loss = 784 * metrics.binary_crossentropy(x, x_recon)
    kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    return recovery_loss + kl_loss

In [ ]:
VAE.compile(loss=vae_loss, optimizer=Adam())
VAE.fit(x_train, x_train, batch_size=32, epochs=100)

Epoch 1/100
44608/60000 [=====================>........] - ETA: 25s - loss: 195.6186

In [ ]:
VAE_Encoder = Model(x, z_mean)

VAE_Encoder.summary()

In [ ]:
h_input = Input(shape=(2,))
VAE_Decoder = Model(h_input, dec_1(dec_2(h_input)))

VAE_Decoder.summary()